# Stock Investment Model Testing

This notebook tests the Stock Investment Evaluation Model implementation.

In [8]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
import sys
from pathlib import Path
import os

# Add project root to path
notebook_path = os.path.abspath('')
project_root = os.path.dirname(os.path.dirname(notebook_path))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import our model
from backend.models.rating_models import StockInvestmentModel

# Set up plotting
plt.style.use('ggplot')
sns.set_theme()
%matplotlib inline

## 1. Generate Stock Data for Testing

In [9]:
def generate_stock_data(ticker='AAPL', period='5y'):
    """
    Generate or fetch stock data for testing.
    
    Args:
        ticker: Stock ticker symbol
        period: Time period to fetch
        
    Returns:
        Dictionary with stock data
    """
    # Fetch data using yfinance
    stock = yf.Ticker(ticker)
    hist = stock.history(period=period)
    
    # Calculate returns
    returns = hist['Close'].pct_change().dropna()
    
    # Get financial data
    try:
        # Try to get real financial data
        info = stock.info
        pe_ratio = info.get('trailingPE', 20.0)
        de_ratio = info.get('debtToEquity', 100.0) / 100.0  # Convert to decimal
        roe = info.get('returnOnEquity', 0.15)
        try:
            fcf = info.get('freeCashflow', 1000000000)
            market_cap = info.get('marketCap', 10000000000)
            fcf_yield = fcf / market_cap
        except:
            fcf_yield = 0.05
        profit_margin = info.get('profitMargin', 0.15)
    except:
        # Use mock data if real data not available
        pe_ratio = 20.0
        de_ratio = 1.0
        roe = 0.15
        fcf_yield = 0.05
        profit_margin = 0.15
    
    # Generate technical indicators
    close_prices = hist['Close']
    volume = hist['Volume']
    
    # Moving averages
    ma_50 = close_prices.rolling(window=50).mean().iloc[-1]
    ma_200 = close_prices.rolling(window=200).mean().iloc[-1]
    
    # RSI (simplified)
    delta = close_prices.diff()
    gain = delta.clip(lower=0).rolling(window=14).mean()
    loss = -delta.clip(upper=0).rolling(window=14).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs.iloc[-1]))
    
    # MACD (simplified)
    ema_12 = close_prices.ewm(span=12, adjust=False).mean()
    ema_26 = close_prices.ewm(span=26, adjust=False).mean()
    macd = ema_12.iloc[-1] - ema_26.iloc[-1]
    macd_signal = close_prices.ewm(span=9, adjust=False).mean().iloc[-1]
    
    # Avg volume
    avg_volume = volume.rolling(window=20).mean().iloc[-1]
    
    technical_indicators = {
        'price': close_prices.iloc[-1],
        'ma_50': ma_50,
        'ma_200': ma_200,
        'rsi': rsi,
        'macd': macd,
        'macd_signal': macd_signal,
        'volume': volume.iloc[-1],
        'avg_volume': avg_volume
    }
    
    # Create stock data dictionary
    stock_data = {
        'ticker': ticker,
        'price_history': hist['Close'],
        'returns': returns,
        'pe_ratio': pe_ratio,
        'de_ratio': de_ratio,
        'roe': roe,
        'fcf_yield': fcf_yield,
        'profit_margin': profit_margin,
        'technical_indicators': technical_indicators
    }
    
    return stock_data

In [10]:
def generate_market_data():
    """
    Generate market data for testing.
    
    Returns:
        Dictionary with market data
    """
    # Fetch S&P 500 data as market proxy
    market = yf.Ticker('^GSPC')
    hist = market.history(period='5y')
    
    # Calculate returns
    market_returns = hist['Close'].pct_change().dropna()
    
    # Current risk-free rate (10-year Treasury yield)
    try:
        treasury = yf.Ticker('^TNX')
        treasury_yield = treasury.history(period='1d')['Close'].iloc[-1] / 100
    except:
        treasury_yield = 0.04  # 4% as fallback
    
    # Sector average ratios
    sector_pe = 20.0
    sector_de = 1.2
    
    # Create market data dictionary
    market_data = {
        'market_returns': market_returns,
        'treasury_yield': treasury_yield,
        'sector_pe': sector_pe,
        'sector_de': sector_de
    }
    
    return market_data

In [11]:
def generate_projections(ticker='AAPL'):
    """
    Generate growth projections for testing.
    
    Args:
        ticker: Stock ticker symbol
        
    Returns:
        Dictionary with growth projections
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        growth = info.get('earningsGrowth', 0.1)
    except:
        growth = 0.1  # 10% as fallback
    
    # Create projections dictionary
    projections = {
        'earnings_growth': growth
    }
    
    return projections

## 2. Test the Model with Different Stocks

In [12]:
def test_model_with_stocks(tickers=['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']):
    """
    Test the model with different stocks.
    
    Args:
        tickers: List of stock ticker symbols
        
    Returns:
        DataFrame with results
    """
    # Initialize model
    model = StockInvestmentModel()
    
    # Generate market data (same for all stocks)
    market_data = generate_market_data()
    
    # Evaluate each stock
    results = []
    for ticker in tickers:
        # Generate stock data
        stock_data = generate_stock_data(ticker)
        
        # Generate growth projections
        projections = generate_projections(ticker)
        
        # Evaluate stock
        evaluation = model.evaluate_stock(stock_data, market_data, projections)
        
        # Add ticker to results
        evaluation['ticker'] = ticker
        results.append(evaluation)
    
    # Convert to DataFrame
    df_results = pd.DataFrame(results)
    
    # Add components as columns
    for component, values in zip(df_results['components'], df_results.index):
        for key, value in component.items():
            df_results.loc[values, key] = value
    
    # Drop the components column
    df_results = df_results.drop('components', axis=1)
    
    return df_results

In [ ]:
# Test with a set of popular stocks
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NFLX', 'NVDA']
results = test_model_with_stocks(tickers)
results

## 3. Visualize Results

In [ ]:
def visualize_results(results_df):
    """
    Visualize model results.
    
    Args:
        results_df: DataFrame with model results
    """
    # Bar chart of investment scores
    plt.figure(figsize=(12, 6))
    sns.barplot(x='ticker', y='investment_score', data=results_df, palette='viridis')
    plt.title('Investment Scores by Stock')
    plt.xlabel('Stock')
    plt.ylabel('Investment Score')
    plt.axhline(y=0.8, color='green', linestyle='--', label='Strong Buy')
    plt.axhline(y=0.65, color='lightgreen', linestyle='--', label='Buy')
    plt.axhline(y=0.5, color='gray', linestyle='--', label='Hold')
    plt.axhline(y=0.35, color='salmon', linestyle='--', label='Reduce')
    plt.legend()
    plt.show()
    
    # Heatmap of components
    components = ['returns_premium', 'growth_projections', 'fundamental_metrics', 
                 'historical_volatility', 'systematic_risk', 'market_sentiment']
    
    plt.figure(figsize=(14, 8))
    component_df = results_df[components].copy()
    component_df.index = results_df['ticker']
    sns.heatmap(component_df, annot=True, cmap='coolwarm', fmt='.3f', linewidths=.5)
    plt.title('Component Values by Stock')
    plt.ylabel('Stock')
    plt.xlabel('Component')
    plt.show()
    
    # Recommendations table
    recommendations = results_df[['ticker', 'investment_score', 'recommendation']]
    recommendations = recommendations.sort_values('investment_score', ascending=False)
    display(recommendations)

# Visualize the results
visualize_results(results)

## 4. Test with Custom Weights

In [ ]:
# Create model with custom weights
custom_model = StockInvestmentModel(
    alpha=0.3,    # Higher weight on historical returns
    beta=0.3,     # Higher weight on growth
    gamma=0.3,    # Higher weight on fundamentals
    delta=0.05,   # Lower weight on volatility
    epsilon=0.05, # Lower weight on beta
    zeta=0.0,     # Ignore market sentiment
    w1=0.1,       # P/E weight
    w2=0.1,       # D/E weight
    w3=0.3,       # ROE weight
    w4=0.3,       # FCF weight
    w5=0.2        # PM weight
)

# Generate results with custom weights
custom_results = []
market_data = generate_market_data()

for ticker in tickers:
    stock_data = generate_stock_data(ticker)
    projections = generate_projections(ticker)
    evaluation = custom_model.evaluate_stock(stock_data, market_data, projections)
    evaluation['ticker'] = ticker
    custom_results.append(evaluation)

# Convert to DataFrame
df_custom = pd.DataFrame(custom_results)

# Add components as columns
for component, values in zip(df_custom['components'], df_custom.index):
    for key, value in component.items():
        df_custom.loc[values, key] = value

# Drop the components column
df_custom = df_custom.drop('components', axis=1)

# Display custom results
print("Results with Custom Weights:")
visualize_results(df_custom)

## 5. Compare Original vs Custom Weights

In [ ]:
# Compare original vs custom
plt.figure(figsize=(12, 6))

# Create a DataFrame for comparison
compare_df = pd.DataFrame({
    'ticker': results['ticker'],
    'Original': results['investment_score'],
    'Custom Weights': df_custom['investment_score']
})

# Reshape for seaborn
compare_df = pd.melt(compare_df, id_vars=['ticker'], var_name='Model', value_name='Score')

# Plot
sns.barplot(x='ticker', y='Score', hue='Model', data=compare_df, palette='Set2')
plt.title('Comparison of Original vs Custom Weights')
plt.xlabel('Stock')
plt.ylabel('Investment Score')
plt.axhline(y=0.8, color='green', linestyle='--', label='Strong Buy')
plt.axhline(y=0.65, color='lightgreen', linestyle='--', label='Buy')
plt.axhline(y=0.5, color='gray', linestyle='--', label='Hold')
plt.axhline(y=0.35, color='salmon', linestyle='--', label='Reduce')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 6. Sensitivity Analysis

In [ ]:
def visualize_sensitivity(model, ticker='AAPL'):
    """
    Visualize sensitivity analysis for a stock.
    
    Args:
        model: StockInvestmentModel instance
        ticker: Stock ticker to analyze
    """
    # Generate data
    stock_data = generate_stock_data(ticker)
    market_data = generate_market_data()
    projections = generate_projections(ticker)
    
    # Define parameter ranges to test
    param_ranges = {
        'alpha': np.linspace(0.1, 0.5, 5),
        'beta': np.linspace(0.1, 0.5, 5),
        'gamma': np.linspace(0.1, 0.5, 5),
        'delta': np.linspace(0.05, 0.25, 5),
        'epsilon': np.linspace(0.05, 0.25, 5),
        'zeta': np.linspace(0.0, 0.2, 5)
    }
    
    # Perform sensitivity analysis
    sensitivity = model.perform_sensitivity_analysis(
        stock_data, market_data, projections, param_ranges
    )
    
    # Visualize results
    plt.figure(figsize=(15, 10))
    
    # Create a subplot for each parameter
    params = sensitivity['parameter'].unique()
    num_params = len(params)
    rows = (num_params + 1) // 2
    
    for i, param in enumerate(params):
        param_data = sensitivity[sensitivity['parameter'] == param]
        
        plt.subplot(rows, 2, i+1)
        sns.lineplot(x='value', y='investment_score', data=param_data, marker='o')
        
        # Add recommendation regions
        plt.axhline(y=0.8, color='green', linestyle='--', alpha=0.4)
        plt.axhline(y=0.65, color='lightgreen', linestyle='--', alpha=0.4)
        plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.4)
        plt.axhline(y=0.35, color='salmon', linestyle='--', alpha=0.4)
        
        plt.title(f'Sensitivity to {param}')
        plt.xlabel(f'{param} Value')
        plt.ylabel('Investment Score')
    
    plt.tight_layout()
    plt.suptitle(f'Sensitivity Analysis for {ticker}', fontsize=16, y=1.02)
    plt.show()
    
    # Show how recommendation changes with parameter values
    plt.figure(figsize=(15, 10))
    for i, param in enumerate(params):
        param_data = sensitivity[sensitivity['parameter'] == param]
        
        plt.subplot(rows, 2, i+1)
        # Convert recommendation to numeric
        recommendation_map = {
            'Strong Buy': 5, 'Buy': 4, 'Hold': 3, 'Reduce': 2, 'Sell': 1
        }
        param_data['rec_num'] = param_data['recommendation'].map(recommendation_map)
        
        # Use step plot to show recommendation changes
        plt.step(param_data['value'], param_data['rec_num'], where='mid')
        plt.yticks([1, 2, 3, 4, 5], ['Sell', 'Reduce', 'Hold', 'Buy', 'Strong Buy'])
        
        plt.title(f'Recommendation Sensitivity to {param}')
        plt.xlabel(f'{param} Value')
        plt.ylabel('Recommendation')
        
    plt.tight_layout()
    plt.suptitle(f'Recommendation Changes for {ticker}', fontsize=16, y=1.02)
    plt.show()

# Run sensitivity analysis for Apple
model = StockInvestmentModel()
visualize_sensitivity(model, 'AAPL')